In [ ]:
%pip install duckdb

In [11]:
import duckdb 
import os
import time

#Importa as Libs e cria o Banco de Dados onde será persistido posteriormente os dados.
con = duckdb.connect(database="ibge.db",read_only=False)
# Apenas para verificar execução.


In [12]:
#Cria a tabela se ela não existir, definida por DATA_IBGE
#COD_UF
#COD_MUN
#COD_ESPECIE
#LATITUDE
#LONGITUDE

con.execute(

"""

  CREATE TABLE IF NOT EXISTS DATA_IBGE (
  COD_UF       INTEGER  NOT NULL
  ,COD_MUN      INTEGER  NOT NULL
  ,COD_ESPECIE  INTEGER  NOT NULL
  ,LATITUDE       NUMERIC(9,6) NOT NULL
  ,LONGITUDE      NUMERIC(10,6) NOT NULL
  ,NV_GEO_COORD INTEGER  NOT NULL
) 
"""
)

In [13]:
#Verificando se a tabela foi criada.
con.sql("SELECT * FROM DATA_IBGE")


┌────────┬─────────┬─────────────┬──────────────┬───────────────┬──────────────┐
│ COD_UF │ COD_MUN │ COD_ESPECIE │   LATITUDE   │   LONGITUDE   │ NV_GEO_COORD │
│ int32  │  int32  │    int32    │ decimal(9,6) │ decimal(10,6) │    int32     │
├────────┼─────────┼─────────────┼──────────────┼───────────────┼──────────────┤
│     17 │ 1718881 │           1 │    -8.771718 │    -47.747535 │            1 │
│     17 │ 1718881 │           1 │    -8.771760 │    -47.747354 │            1 │
│     17 │ 1718881 │           1 │    -8.771843 │    -47.747104 │            1 │
│     17 │ 1718881 │           1 │    -8.772038 │    -47.746765 │            1 │
│     17 │ 1718881 │           1 │    -8.772228 │    -47.746795 │            1 │
│     17 │ 1718881 │           6 │    -8.772457 │    -47.746936 │            1 │
│     17 │ 1718881 │           1 │    -8.772422 │    -47.747123 │            1 │
│     17 │ 1718881 │           1 │    -8.772378 │    -47.747278 │            1 │
│     17 │ 1718881 │        

```
Uma outra possibilidade, seria utilizando o pyarrow para processamento, localizando os arquivos localmente, salvando-os em parquet, 
mas para esta solução, optamos pelo DuckDB.


import pyarrow as pa
import pyarrow.csv as pacsv
import pyarrow.parquet as papq
import os
# Especifique o diretório que você deseja listar
path = os.getcwd()
# Lista todos os arquivos e diretórios no diretório especificado
arquivo = os.listdir(path)

database_path = r"C:\Users\wilke\OneDrive\Área de Trabalho\Unifor\Engenharia de dados\IBGE\Pipe\database"
tables = []

for arquivos in arquivo:
    if arquivos.endswith('.csv'):
        files_path = os.path.join(path, arquivos)
        try:
            table = pacsv.read_csv(files_path)
            tables.append(table)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")

temp = r"C:\Users\wilke\OneDrive\Área de Trabalho\Unifor\Engenharia de dados\IBGE\Pipe\database\ibge.parquet"

database = pa.concat_tables(tables)

papq.write_table(database,temp,compression='snappy')
        
print("Conversão concluída.")
```
      

In [14]:
#Obtendo caminho local
#Uma outra possibilidade seria a solução também, ser utilizada lendo diretamente em outros serviços, o google collab para fins de estudos poderia ler diretamente no google drive, fazendo a conexão com o google drive ou demais fontes conectoras em nuvem.
#No caso utilizei a leitura local.

path = os.getcwd()

print(path)



c:\Users\wilke\OneDrive\Área de Trabalho\Unifor\Engenharia de dados\IBGE\Pipe


In [15]:
%%time

con.sql(f"COPY (SELECT * FROM '{path}/*.csv') TO 'data'   (FORMAT PARQUET,PER_THREAD_OUTPUT true, COMPRESSION Snappy,OVERWRITE_OR_IGNORE 1, ROW_GROUP_SIZE 100_000);")
#salvando em parquet no folder data, onde poderia ser uma outra camada do processo, dependendo da arquitetura, parametros de encoding viabilizados pela documentação


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 1min 7s
Wall time: 20.3 s


No nosso projeto, estamos dividindo os parquets em 7 partes, isto pode ser utilizado com outras estratégias de armazenamento como no exemplo a seguir:

```bash
├── year=2021
│ ├── month=1
│ │ ├── data_1.parquet
│ │ └── data_2.parquet
│ └── month=2
│ └── data_1.parquet
└── year=2022
├── month=11
│ ├── data_1.parquet
│ └── data_2.parquet
└── month=12
└── data_1.parquet
```

In [16]:
%%time
con.sql(f"INSERT INTO DATA_IBGE SELECT * FROM '{path}/data/*.parquet'")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 49.9 s
Wall time: 12.2 s


In [17]:
%%time
con.sql("SELECT COUNT(*)  AS TOTAL_DE_LINHAS FROM DATA_IBGE")
#Verificando o total de linhas inseridas


CPU times: total: 0 ns
Wall time: 964 µs


┌─────────────────┐
│ TOTAL_DE_LINHAS │
│      int64      │
├─────────────────┤
│       222205750 │
└─────────────────┘

In [18]:
%%time
con.sql('SELECT COD_UF as UF,COUNT(COD_UF) AS Total_de_Registros FROM DATA_IBGE GROUP BY COD_UF')


CPU times: total: 0 ns
Wall time: 0 ns


┌───────┬────────────────────┐
│  UF   │ Total_de_Registros │
│ int32 │       int64        │
├───────┼────────────────────┤
│    11 │            1930740 │
│    12 │             821048 │
│    13 │            3317940 │
│    14 │             521030 │
│    15 │            7822340 │
│    16 │             625330 │
│    17 │            1696874 │
│    21 │            6515686 │
│    22 │            3782842 │
│    23 │            9501284 │
│     · │               ·    │
│     · │               ·    │
│     · │               ·    │
│    32 │            4442696 │
│    33 │           17924400 │
│    35 │           45907450 │
│    41 │           12244050 │
│    42 │            8362278 │
│    43 │           12913494 │
│    50 │            3015638 │
│    51 │            3970628 │
│    52 │            7921874 │
│    53 │            2637774 │
├───────┴────────────────────┤
│     27 rows (20 shown)     │
└────────────────────────────┘

In [19]:
con.close()




In [20]:
# Utilizado a documentação do Duck DB referenciada: https://duckdb.org/docs/guides/index
# Parquet Files https://duckdb.org/docs/data/partitioning/partitioned_writes
# How to head CSV Files https://duckdb.org/docs/guides/import/csv_import
